In [22]:
import os

MODEL = "gemma3"
MODEL_EMBED = "embeddinggemma"


In [6]:
from langchain_ollama import OllamaLLM

model = Ollama(model=MODEL)

model.invoke('tell me a joke')

'Why did the scarecrow win an award? \n\n... Because he was outstanding in his field! 😄 \n\n---\n\nWould you like to hear another one?'

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resume2025.pdf")
pages = loader.load_and_split()


In [10]:
from langchain.prompts import PromptTemplate

template = '''
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
'''

prompt = PromptTemplate.from_template(template)
prompt.format(context="I need a Data Scientist", question="Is this a good fit")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: I need a Data Scientist\n\nQuestion: Is this a good fit\n'

In [12]:
chain = prompt | model 

In [13]:
chain.input_schema.schema()

/var/folders/z2/qtr66ng91gj6czktr43rbl4h0000gn/T/ipykernel_33505/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [11]:
chain.invoke(
    {
        'context': 'I need a data scientist who knows python',
        'question': 'what do i need'
    }
)

'You need a data scientist who knows Python.'

In [23]:
from langchain_community.vectorstores import DocArrayInMemorySearch # do I need permanent storage like pinecone?
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL_EMBED)

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [25]:
retriever = vectorstore.as_retriever()

In [31]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    
)

print(chain.invoke({"question": "what soft skills are evident"}))
print(chain.invoke({"question": "would she be a good fit for our data scientist role"}))

Based on the provided document, the following soft skills are evident:

*   **Cross-functional Leadership:** Demonstrated in her roles at Booz Allen Hamilton and Jovial.
*   **Stakeholder Management:** Highlighted in multiple experiences, including client relationships at Jovial and collaboration with SMEs at Booz Allen.
*   **Technical Writing & Editing:** Mentioned as a key skill and evident in her roles at Jovial and as a contract scientific editor.
*   **Communication:** Clear from her presentations, technical writing, and ability to explain complex concepts to non-technical audiences.
*   **Teamwork/Collaboration:** Working with SMEs, researchers, developers, and students suggests a collaborative approach.
*   **Presentation Skills:** Explicitly mentioned as a key skill, and demonstrated through her capstone project and other roles.
